In [ ]:
# Install dependencies (run this cell first!)
!uv pip install -q "pydantic-ai-slim[logfire,openai,tavily,a2a]>=0.0.49" "pydantic-settings>=2.0.0" "logfire[httpx]>=4.3.3" "httpx>=0.27.0"
print("✅ Dependencies installed successfully!")

# Lecture 02: Single Agent Stock Analysis

This notebook demonstrates a single AI agent that uses web search to analyze stocks and generate comprehensive reports.


In [ ]:
# Setup: Import necessary modules
from pydantic_ai import Agent
from common.models import StockReport
from common.tools import web_search_tool
from common.utils import create_agent_model, setup_logging, get_current_date

# Initialize logging
setup_logging()

In [ ]:
# Add Logfire setup
from common.utils import setup_logfire

setup_logfire(
    service_name="stock-analysis-lecture02",
    start_message="🚀 Lecture 02 - Stock Analysis Notebook Started",
)

In [ ]:
# Create the stock analysis agent with structured output
stock_analysis_agent = Agent(
    model=create_agent_model(),
    tools=[web_search_tool],
    output_type=StockReport,  # Returns structured StockReport
    system_prompt="""
You are an expert financial analyst specializing in stock analysis and market research.

Your task is to create comprehensive stock reports by gathering and analyzing current information.

When analyzing a stock, you should:
1. **Company Overview**: Get basic company information and recent developments
2. **Financial Performance**: Look for recent earnings, revenue trends, and key metrics
3. **Market Position**: Research competitive landscape and market share
4. **Recent News**: Find recent news that could impact the stock price
5. **Market Sentiment**: Gauge overall market sentiment toward the stock
6. **Risk Assessment**: Identify potential risks and challenges

Provide a structured analysis with specific data points when available.
""".strip(),
)

In [ ]:
# Analyze a stock
stock_symbol = "AAPL"
print(f"Analyzing {stock_symbol}...\n")

prompt = f"Please analyze the stock {stock_symbol}. The analysis date should be: {get_current_date()}"
result = await stock_analysis_agent.run(prompt)
report = result.output

# Display the report
print(f"Stock Report for {report.symbol}")
print(f"Company: {report.company_name}")
print(f"Analysis Date: {report.analysis_date}")
print(f"\nCurrent Price: ${report.current_price:.2f}")
print(f"52-Week Range: ${report.week_52_low:.2f} - ${report.week_52_high:.2f}")
print(f"Market Cap: ${report.market_cap:,.0f}")
print(f"P/E Ratio: {report.pe_ratio:.2f}")
print(f"Dividend Yield: {report.dividend_yield:.2%}")
print(f"\nSummary: {report.summary}")
print(f"\nRecommendation: {report.recommendation}")

In [ ]:
# Try another stock
stock_symbol = "TSLA"
print(f"Analyzing {stock_symbol}...\n")

prompt = f"Please analyze the stock {stock_symbol}. The analysis date should be: {get_current_date()}"
result = await stock_analysis_agent.run(prompt)
report = result.output

print(f"Stock Report for {report.symbol}")
print(f"Company: {report.company_name}")
print(f"Current Price: ${report.current_price:.2f}")
print(f"\nSummary: {report.summary}")
print(f"\nRecommendation: {report.recommendation}")